<a href="https://colab.research.google.com/github/benpettis/DS-Unit-2-Linear-Models/blob/master/module2-regression-2/Ben_Pettis_LS_DS14_212_assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Lambda School Data Science

*Unit 2, Sprint 1, Module 2*

---

# Regression 2

## Assignment

You'll continue to **predict how much it costs to rent an apartment in NYC,** using the dataset from renthop.com.

- [ ] Do train/test split. Use data from April & May 2016 to train. Use data from June 2016 to test.
- [ ] Engineer at least two new features. (See below for explanation & ideas.)
- [ ] Fit a linear regression model with at least two features.
- [ ] Get the model's coefficients and intercept.
- [ ] Get regression metrics RMSE, MAE, and $R^2$, for both the train and test data.
- [ ] What's the best test MAE you can get? Share your score and features used with your cohort on Slack!
- [ ] As always, commit your notebook to your fork of the GitHub repo.


#### [Feature Engineering](https://en.wikipedia.org/wiki/Feature_engineering)

> "Some machine learning projects succeed and some fail. What makes the difference? Easily the most important factor is the features used." — Pedro Domingos, ["A Few Useful Things to Know about Machine Learning"](https://homes.cs.washington.edu/~pedrod/papers/cacm12.pdf)

> "Coming up with features is difficult, time-consuming, requires expert knowledge. 'Applied machine learning' is basically feature engineering." — Andrew Ng, [Machine Learning and AI via Brain simulations](https://forum.stanford.edu/events/2011/2011slides/plenary/2011plenaryNg.pdf) 

> Feature engineering is the process of using domain knowledge of the data to create features that make machine learning algorithms work. 

#### Feature Ideas
- Does the apartment have a description?
- How long is the description?
- How many total perks does each apartment have?
- Are cats _or_ dogs allowed?
- Are cats _and_ dogs allowed?
- Total number of rooms (beds + baths)
- Ratio of beds to baths
- What's the neighborhood, based on address or latitude & longitude?

## Stretch Goals
- [ ] If you want more math, skim [_An Introduction to Statistical Learning_](http://faculty.marshall.usc.edu/gareth-james/ISL/ISLR%20Seventh%20Printing.pdf),  Chapter 3.1, Simple Linear Regression, & Chapter 3.2, Multiple Linear Regression
- [ ] If you want more introduction, watch [Brandon Foltz, Statistics 101: Simple Linear Regression](https://www.youtube.com/watch?v=ZkjP5RJLQF4)
(20 minutes, over 1 million views)
- [ ] Add your own stretch goal(s) !

In [0]:
%%capture
import sys

# If you're on Colab:
if 'google.colab' in sys.modules:
    DATA_PATH = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Applied-Modeling/master/data/'
    !pip install category_encoders==2.*

# If you're working locally:
else:
    DATA_PATH = '../data/'
    
# Ignore this Numpy warning when using Plotly Express:
# FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
import warnings
warnings.filterwarnings(action='ignore', category=FutureWarning, module='numpy')

In [0]:
import numpy as np
import pandas as pd

# Read New York City apartment rental listing data
df = pd.read_csv(DATA_PATH+'apartments/renthop-nyc.csv')
assert df.shape == (49352, 34)

# Remove the most extreme 1% prices,
# the most extreme .1% latitudes, &
# the most extreme .1% longitudes
df = df[(df['price'] >= np.percentile(df['price'], 0.5)) & 
        (df['price'] <= np.percentile(df['price'], 99.5)) & 
        (df['latitude'] >= np.percentile(df['latitude'], 0.05)) & 
        (df['latitude'] < np.percentile(df['latitude'], 99.95)) &
        (df['longitude'] >= np.percentile(df['longitude'], 0.05)) & 
        (df['longitude'] <= np.percentile(df['longitude'], 99.95))]

In [0]:
# change 'created' column to date format
df['created'] = pd.to_datetime(df['created'])

In [291]:
# check to see if any values in 'created' are before april
df.sort_values(by='created')

,bathrooms,bedrooms,created,description,display_address,latitude,longitude,price,street_address,interest_level,elevator,cats_allowed,hardwood_floors,dogs_allowed,doorman,dishwasher,no_fee,laundry_in_building,fitness_center,pre-war,laundry_in_unit,roof_deck,outdoor_space,dining_room,high_speed_internet,balcony,swimming_pool,new_construction,terrace,exclusive,loft,garden_patio,wheelchair_access,common_outdoor_space
5186,1.0,1,2016-04-01 22:12:41,X-LARGE Flex 1BR Loft! ~~ PRIME Greenwich Vill...,Astor Place,40.7302,-73.9924,3195,1 Astor Place,high,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
7945,1.0,0,2016-04-01 22:56:00,"This Enormous Studio Features: Harwood Floors,...",East 54th Street,40.7576,-73.9677,2000,230 East 54th Street,medium,0,1,0,1,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
6424,2.0,3,2016-04-01 22:57:15,--- East 31st St & Lexington Avenue --- This S...,East 31st St & Lexington Avenue,40.7388,-73.9851,5850,105 Lexington Avenue,high,1,1,0,1,1,0,1,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1
7719,1.0,1,2016-04-01 23:26:07,Reduced Fee!! Priced To Rent!\rLarge Newly Upd...,West End Ave,40.7939,-73.9738,2745,700 West End Ave,medium,1,1,0,1,1,0,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1
1723,1.0,1,2016-04-02 00:48:13,Phenomenal deal of the century!! This spacious...,E 88th street,40.7784,-73.9491,2400,401 E 88th street,medium,1,1,0,1,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19943,1.0,2,2016-06-29 17:47:34,"Newly renovated, clean, quiet & very bright 2...",At Thompson St,40.7261,-74.0014,3045,132 Thompson St #20,medium,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
16801,1.0,1,2016-06-29 17:56:12,Historic conversion in Greenpoint with awesome...,"100 Dupont St, Brooklyn, NY 11222",40.7358,-73.9560,2648,100 Dupont St,low,1,1,0,1,0,1,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
32633,1.0,1,2016-06-29 18:14:48,"Large, sunny one bedroom apartment with a sepa...",West 45th St. and 8th Ave.,40.7597,-73.9890,2650,305 West 45th St.,low,1,1,0,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
20560,1.0,3,2016-06-29 18:30:41,LOOK NO FURTHER!!\r\rMassive three bedroom apa...,williamsburg,40.7156,-73.9540,3200,32 Havemeyer Street,medium,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [0]:
# create feature for whether or not both cats & dogs are allowed
def catdog(data):
  if data['cats_allowed'] == 1 and data['dogs_allowed'] == 1:
    val = 1
  else:
    val = 0
  return val
  

In [0]:
df['catdog_allowed'] = df.apply(catdog, axis=1)

In [0]:
# create feature counting number of perks
df['perkiness'] = df['elevator'] + df['dishwasher'] + df['fitness_center'] + df['laundry_in_unit'] + df['swimming_pool'] + df['terrace'] + df['balcony'] + df['common_outdoor_space'] + df['doorman'] + df['laundry_in_building'] + df['high_speed_internet'] + df['outdoor_space'] + df['laundry_in_building'] + df['roof_deck'] + df['garden_patio'] + df['hardwood_floors']

In [0]:
# split data into train (april & may 2016) and test (june 2016-)
train = df[df['created'] < '2016-06-01']
test = df[df['created'] >= '2016-06-01']

In [296]:
train.head()

,bathrooms,bedrooms,created,description,display_address,latitude,longitude,price,street_address,interest_level,elevator,cats_allowed,hardwood_floors,dogs_allowed,doorman,dishwasher,no_fee,laundry_in_building,fitness_center,pre-war,laundry_in_unit,roof_deck,outdoor_space,dining_room,high_speed_internet,balcony,swimming_pool,new_construction,terrace,exclusive,loft,garden_patio,wheelchair_access,common_outdoor_space,catdog_allowed,perkiness
2,1.0,1,2016-04-17 03:26:41,"Top Top West Village location, beautiful Pre-w...",W 13 Street,40.7388,-74.0018,2850,241 W 13 Street,high,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4
3,1.0,1,2016-04-18 02:22:02,Building Amenities - Garage - Garden - fitness...,East 49th Street,40.7539,-73.9677,3275,333 East 49th Street,low,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
4,1.0,4,2016-04-28 01:32:41,Beautifully renovated 3 bedroom flex 4 bedroom...,West 143rd Street,40.8241,-73.9493,3350,500 West 143rd Street,low,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5,2.0,4,2016-04-19 04:24:47,,West 18th Street,40.7429,-74.0028,7995,350 West 18th Street,medium,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
6,1.0,2,2016-04-27 03:19:56,Stunning unit with a great location and lots o...,West 107th Street,40.8012,-73.9660,3600,210 West 107th Street,low,0,1,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0


In [297]:
# linear regression with multiple features
from sklearn.linear_model import LinearRegression
# instantiate class
model = LinearRegression()
# select x features and fit model
features = ['bedrooms', 'longitude', 'perkiness', 'catdog_allowed']
X_train = train[features]
X_test = test[features]
target = 'price'
y_train = train[target]
y_test = test[target]
model.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [298]:
# get model coef and intercept
print('Intercept', model.intercept_)
coefficients = pd.Series(model.coef_, features)
print(coefficients.to_string())

Intercept -1028288.3210185637
bedrooms            845.759432
longitude        -13926.305152
perkiness           114.790804
catdog_allowed      126.218710


In [299]:
# get mae
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
y_pred_train = model.predict(X_train)
mae = mean_absolute_error(y_train, y_pred_train)
print(f'Train MAE: {mae:.2f}')
y_pred_test = model.predict(X_test)
mae = mean_absolute_error(y_test, y_pred_test)
print(f'Test MAE: {mae:.2f}')


Train MAE: 855.65
Test MAE: 863.13


In [300]:
# get rmse
mse = mean_squared_error(y_train, y_pred_train)
rmse = np.sqrt(mse)
print(f'Train RMSE: {rmse:.2f}')
mse = mean_squared_error(y_test, y_pred_test)
rmse = np.sqrt(mse)
print(f'Test RMSE: {rmse:.2f}')

Train RMSE: 1367.52
Test RMSE: 1367.01


In [301]:
# get r^2 value
r2 = r2_score(y_train, y_pred_train)
print(f'Train R^2: {r2:.5f}')
r2 = r2_score(y_test, y_pred_test)
print(f'Test R^2: {r2:.5f}')

Train R^2: 0.39772
Test R^2: 0.39875


In [309]:
# try to get lower MAE 
# checking all features for lowest MAE
cols = list(df._get_numeric_data())
for col in cols:
  model = LinearRegression()
  X_train = train[[col]]
  X_test = test[[col]]
  target = 'price'
  y_train = train[target]
  y_test = test[target]
  model.fit(X_train, y_train)
  y_pred_test = model.predict(X_test)
  mae = mean_absolute_error(y_test, y_pred_test)
  print(f'{col} test MAE: {mae:.2f}')


bathrooms test MAE: 890.46
bedrooms test MAE: 988.73
latitude test MAE: 1195.62
longitude test MAE: 1138.53
price test MAE: 0.00
elevator test MAE: 1163.71
cats_allowed test MAE: 1194.60
hardwood_floors test MAE: 1195.37
dogs_allowed test MAE: 1194.04
doorman test MAE: 1143.44
dishwasher test MAE: 1169.04
no_fee test MAE: 1189.65
laundry_in_building test MAE: 1198.31
fitness_center test MAE: 1163.70
pre-war test MAE: 1196.74
laundry_in_unit test MAE: 1165.43
roof_deck test MAE: 1188.83
outdoor_space test MAE: 1179.89
dining_room test MAE: 1162.07
high_speed_internet test MAE: 1191.62
balcony test MAE: 1182.06
swimming_pool test MAE: 1188.18
new_construction test MAE: 1197.21
terrace test MAE: 1182.90
exclusive test MAE: 1197.43
loft test MAE: 1197.71
garden_patio test MAE: 1189.08
wheelchair_access test MAE: 1195.58
common_outdoor_space test MAE: 1197.32
catdog_allowed test MAE: 1194.03
perkiness test MAE: 1126.85


In [312]:

# linear regression with multiple features
from sklearn.linear_model import LinearRegression
# instantiate class
model = LinearRegression()
# select x features and fit model
features = ['bedrooms', 'bathrooms', 'longitude', 'doorman', 'catdog_allowed', 'perkiness','dishwasher']
X_train = train[features]
X_test = test[features]
target = 'price'
y_train = train[target]
y_test = test[target]
model.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [313]:
# predict new values & get MAE with new features
y_pred_train = model.predict(X_train)
mae = mean_absolute_error(y_train, y_pred_train)
print(f'Train MAE: {mae:.2f}')
y_pred_test = model.predict(X_test)
mae = mean_absolute_error(y_test, y_pred_test)
print(f'Test MAE: {mae:.2f}')

Train MAE: 706.57
Test MAE: 712.80
